

```
Copyright © 2025 by Sonain Jamil and Yaseen. All rights reserved.
```



**Install Dependencies**

In [ ]:
!pip install numpy scipy trimesh open3d pyrender scikit-image scikit-learn
!pip install torch torchvision lpips

**Upload Meshes from PC**

In [ ]:
from google.colab import files
uploaded = files.upload()

**Import necessary libraries**

In [ ]:
import open3d as o3d
import numpy as np
import trimesh
from scipy.spatial import cKDTree

**Utility Functions**

In [ ]:
# ===== Utility functions =====
def sample_points(mesh, n=30000):
    pts, face_idx = trimesh.sample.sample_surface(mesh, n)
    normals = mesh.face_normals[face_idx]
    return pts, normals

1. Chamfer Distance (CD)

In [ ]:
def chamfer(mesh_a, mesh_b, samples=30000):
    pa, _ = sample_points(mesh_a, samples)
    pb, _ = sample_points(mesh_b, samples)
    ka, kb = cKDTree(pa), cKDTree(pb)
    da,_ = kb.query(pa, k=1)
    db,_ = ka.query(pb, k=1)
    return da.mean(), db.mean(), (da.mean()+db.mean())/2

2. Hausdorff Distance (HD)

In [ ]:
def hausdorff(mesh_a, mesh_b, samples=30000):
    pa,_ = sample_points(mesh_a, samples)
    pb,_ = sample_points(mesh_b, samples)
    ka, kb = cKDTree(pa), cKDTree(pb)
    da,_ = kb.query(pa, k=1)
    db,_ = ka.query(pb, k=1)
    return da.max(), db.max(), max(da.max(), db.max())

3. RMS Vertex Distance

In [ ]:
def rms_vertex_distance_o3d(mesh_a, mesh_b):
    a = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(np.asarray(mesh_a.vertices)),
        triangles=o3d.utility.Vector3iVector(np.asarray(mesh_a.faces))
    )
    b = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(np.asarray(mesh_b.vertices)),
        triangles=o3d.utility.Vector3iVector(np.asarray(mesh_b.faces))
    )
    a.compute_vertex_normals()
    b.compute_vertex_normals()

    # use point-to-mesh distance by sampling surface of B
    pcd_B = b.sample_points_poisson_disk(50000)
    pts_B = np.asarray(pcd_B.points)

    from sklearn.neighbors import KDTree
    tree = KDTree(pts_B)

    verts_A = np.asarray(mesh_a.vertices)
    dist, _ = tree.query(verts_A, k=1)

    return float(np.sqrt((dist**2).mean()))

**Load a pair of mesh**

In [ ]:
# ===== Load your two meshes =====
A = trimesh.load("/content/01.stl", process=True)
B = trimesh.load("/content/01_m.stl", process=True)

**Evalute 3D metrics and print results**

In [ ]:
print("=== 3D METRIC RESULTS ===\n")
print("Chamfer Distance (A→B, B→A, sym):")
print(chamfer(A, B))
print("\nHausdorff Distance (A→B, B→A, sym):")
print(hausdorff(A, B))
print("\nRMS Vertex Distance (Open3D):")
print(rms_vertex_distance_o3d(A, B))